In [5]:
import yfinance as yf
import pandas as pd
import numpy as np

# Parameters
ticker_symbol = "AAPL"
start = "2023-01-01"
end = "2024-01-01"

# Get historical EOD price data
ticker = yf.Ticker(ticker_symbol)
historical_data = ticker.history(start=start, end=end)
closing_prices = historical_data['Close']
today_price = closing_prices.iloc[-1]

# Get list of available expirations
option_expirations = ticker.options

# Compute volatility from historical data
log_returns = np.log(closing_prices / closing_prices.shift(1)).dropna()
daily_volatility = log_returns.std()
annualized_volatility = daily_volatility * np.sqrt(252)

# Constants for pricing
r = 0.05  # Risk-free interest rate
N = 50    # Increased number of steps for more accurate binomial tree
sigma = annualized_volatility

# Function: Binomial Option Pricing Model (European Call)
def binomial_option_price(S0, K, T, r, sigma, N, option_type='call'):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    # Initialize asset prices at maturity
    ST = np.array([S0 * (u**j) * (d**(N - j)) for j in range(N + 1)])

    # Payoffs at maturity
    if option_type == 'call':
        option_values = np.maximum(0, ST - K)
    elif option_type == 'put':
        option_values = np.maximum(0, K - ST)

    # Backward induction
    for i in range(N - 1, -1, -1):
        option_values = np.exp(-r * dt) * (p * option_values[1:] + (1 - p) * option_values[:-1])

    return option_values[0]

# Backtest over last 5 available expirations
backtest_results = []

from datetime import datetime as dt_now

for expiry_date in option_expirations[-5:]:
    try:
        expiry_datetime = pd.to_datetime(expiry_date)
        T = (expiry_datetime - pd.Timestamp(dt_now.now().date())).days / 365

        if T <= 0:
            continue  # Expired

        # Get option chain for this expiry
        options_chain = ticker.option_chain(expiry_date)
        calls = options_chain.calls

        # Use the strike closest to the current price
        nearest_call = calls.iloc[(calls['strike'] - today_price).abs().argmin()]
        strike = nearest_call['strike']
        market_price = nearest_call['lastPrice']

        # Compute model price
        model_price = binomial_option_price(today_price, strike, T, r, sigma, N, option_type='call')

        # Record results
        backtest_results.append({
            "expiry": expiry_date,
            "strike": round(float(strike), 2),
            "market_price": round(float(market_price), 2),
            "model_price": round(float(model_price), 2),
            "error": round(model_price - market_price, 2),
            "relative_error_%": round((model_price - market_price) / market_price * 100, 2)
        })

    except Exception as e:
        print(f"Skipping expiry {expiry_date} due to error: {e}")

# Output: DataFrame of results
backtest_df = pd.DataFrame(backtest_results)
print(backtest_df)

       expiry  strike  market_price  model_price  error  relative_error_%
0  2026-09-18   190.0         41.44        22.49 -18.95            -45.73
1  2026-12-18   190.0         44.68        25.29 -19.39            -43.40
2  2027-01-15   190.0         45.28        26.12 -19.16            -42.31
3  2027-06-17   190.0         49.26        30.43 -18.83            -38.22
4  2027-12-17   190.0         53.90        35.21 -18.69            -34.67
